In [1]:
import pandas as pd
from sqlalchemy import create_engine, func
from  db_conn import url
import psycopg2

In [2]:
postgresInfo = f'postgresql://{url}'
conn_string = postgresInfo

db = create_engine(conn_string)
conn = db.connect()

conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()

In [3]:
concussed_df = pd.read_sql_query('''SELECT * FROM clean_concussion_data ;''', db)
stats_df = pd.read_sql_query(''' SELECT * FROM moneypuck_cleaned; ''', db)

In [4]:
concussed_df.head()

,Season,Team,Position,Player,Games_Missed
0,2012,BOS,F,Patrice Bergeron,6
1,2012,BOS,F,Brad Marchand,2
2,2012,BOS,F,Shawn Thornton,2
3,2012,CAR,F,Chad LaRose,10
4,2012,CAR,F,Alexander Semin,2


In [5]:
stats_df.head()

,season,name,team,position,games_played,onIce_corsiPercentage,onIce_fenwickPercentage,I_F_goals,I_F_points,I_F_penalityMinutes,I_F_hits,I_F_takeaways,I_F_giveaways
0,2012,Ryan Johansen,CBJ,C,40,0.51,0.50,5.0,12.0,12.0,42.0,15.0,6.0
1,2012,Stephen Weiss,FLA,C,17,0.45,0.43,1.0,4.0,15.0,6.0,13.0,2.0
2,2012,Anton Babchuk,CGY,D,7,0.41,0.41,0.0,1.0,0.0,1.0,0.0,5.0
3,2012,Frans Nielsen,NYI,C,48,0.48,0.48,6.0,29.0,12.0,23.0,29.0,24.0
4,2012,Kyle Quincey,DET,D,36,0.53,0.53,1.0,3.0,18.0,29.0,6.0,18.0


In [6]:
concussed_df = concussed_df.rename(columns={'Season': 'season'})
concussed_df = concussed_df.rename(columns={'Player': 'name'})


In [7]:
concussed_df.head()

,season,Team,Position,name,Games_Missed
0,2012,BOS,F,Patrice Bergeron,6
1,2012,BOS,F,Brad Marchand,2
2,2012,BOS,F,Shawn Thornton,2
3,2012,CAR,F,Chad LaRose,10
4,2012,CAR,F,Alexander Semin,2


In [8]:
concussed_df.value_counts("Games_Missed")

Games_Missed
3     48
4     44
2     36
5     26
6     26
1     21
9     20
7     17
10    16
8     13
12     9
11     8
15     6
14     6
18     5
16     4
19     4
21     4
17     3
20     3
25     3
27     2
56     2
42     2
78     2
34     2
24     2
13     2
23     2
26     2
30     1
22     1
81     1
80     1
74     1
69     1
62     1
49     1
50     1
48     1
46     1
44     1
41     1
40     1
35     1
82     1
dtype: int64

In [9]:
stats_df.shape

(9938, 13)

In [10]:
filtered_concussed_df = concussed_df[concussed_df['Games_Missed'] >= 10 ]
filtered_concussed_df.value_counts("Games_Missed")

Games_Missed
10    16
12     9
11     8
14     6
15     6
18     5
16     4
19     4
21     4
17     3
20     3
25     3
27     2
42     2
78     2
34     2
23     2
26     2
24     2
13     2
56     2
50     1
80     1
81     1
74     1
69     1
62     1
30     1
49     1
48     1
46     1
44     1
41     1
40     1
35     1
22     1
82     1
dtype: int64

In [11]:
filtered_concussed_df.shape

(105, 5)

In [12]:
filtered_concussed_df.value_counts("name")

name
Ondrej Kase           4
Clarke MacArthur      3
Andrew Shaw           3
Patrick Sharp         2
George Parros         2
                     ..
Jonathan Huberdeau    1
John-Michael Liles    1
John Moore            1
Jesse Winchester      1
Vladimir Tarasenko    1
Length: 85, dtype: int64

In [13]:
merged_df = pd.merge(stats_df, filtered_concussed_df, on=['season', 'name'], how='left').fillna(0)
merged_df.head()


,season,name,team,position,games_played,onIce_corsiPercentage,onIce_fenwickPercentage,I_F_goals,I_F_points,I_F_penalityMinutes,I_F_hits,I_F_takeaways,I_F_giveaways,Team,Position,Games_Missed
0,2012,Ryan Johansen,CBJ,C,40,0.51,0.50,5.0,12.0,12.0,42.0,15.0,6.0,0,0,0.0
1,2012,Stephen Weiss,FLA,C,17,0.45,0.43,1.0,4.0,15.0,6.0,13.0,2.0,0,0,0.0
2,2012,Anton Babchuk,CGY,D,7,0.41,0.41,0.0,1.0,0.0,1.0,0.0,5.0,0,0,0.0
3,2012,Frans Nielsen,NYI,C,48,0.48,0.48,6.0,29.0,12.0,23.0,29.0,24.0,0,0,0.0
4,2012,Kyle Quincey,DET,D,36,0.53,0.53,1.0,3.0,18.0,29.0,6.0,18.0,0,0,0.0


In [14]:
final_df = merged_df.drop(['Team', 'Position'], axis = 1)
final_df.head()

,season,name,team,position,games_played,onIce_corsiPercentage,onIce_fenwickPercentage,I_F_goals,I_F_points,I_F_penalityMinutes,I_F_hits,I_F_takeaways,I_F_giveaways,Games_Missed
0,2012,Ryan Johansen,CBJ,C,40,0.51,0.50,5.0,12.0,12.0,42.0,15.0,6.0,0.0
1,2012,Stephen Weiss,FLA,C,17,0.45,0.43,1.0,4.0,15.0,6.0,13.0,2.0,0.0
2,2012,Anton Babchuk,CGY,D,7,0.41,0.41,0.0,1.0,0.0,1.0,0.0,5.0,0.0
3,2012,Frans Nielsen,NYI,C,48,0.48,0.48,6.0,29.0,12.0,23.0,29.0,24.0,0.0
4,2012,Kyle Quincey,DET,D,36,0.53,0.53,1.0,3.0,18.0,29.0,6.0,18.0,0.0


In [15]:
final_df.value_counts('Games_Missed')

Games_Missed
0.0     9849
10.0      16
12.0       9
11.0       7
14.0       6
15.0       6
18.0       5
21.0       4
16.0       4
17.0       3
20.0       3
25.0       3
19.0       3
78.0       2
24.0       2
13.0       2
27.0       2
34.0       2
42.0       2
50.0       1
80.0       1
74.0       1
69.0       1
62.0       1
56.0       1
26.0       1
46.0       1
44.0       1
41.0       1
40.0       1
35.0       1
23.0       1
22.0       1
81.0       1
dtype: int64

In [16]:
final_df.shape


(9945, 14)

In [17]:
filtered_concussed_df.shape

(105, 5)

In [18]:
has_missed_game = final_df.groupby('name')['Games_Missed'].max() > 0
df_final = final_df[final_df['name'].isin(has_missed_game[has_missed_game].index)]

In [19]:
df_final.shape

(579, 14)

In [20]:
df_final['points_per_game'] = df_final['I_F_points'] / df_final['games_played']
df_final.head()

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,season,name,team,position,games_played,onIce_corsiPercentage,onIce_fenwickPercentage,I_F_goals,I_F_points,I_F_penalityMinutes,I_F_hits,I_F_takeaways,I_F_giveaways,Games_Missed,points_per_game
40,2012,Jack Hillen,WSH,D,23,0.49,0.50,3.0,9.0,14.0,10.0,2.0,13.0,0.0,0.391304
56,2012,David Perron,STL,L,48,0.58,0.58,10.0,25.0,44.0,48.0,26.0,14.0,0.0,0.520833
94,2012,Ryane Clowe,NYR,L,40,0.58,0.58,3.0,19.0,53.0,99.0,13.0,26.0,0.0,0.475000
103,2012,Nicklas Grossmann,PHI,D,30,0.41,0.42,1.0,4.0,21.0,81.0,6.0,17.0,18.0,0.133333
117,2012,Vladimir Tarasenko,STL,R,38,0.63,0.63,8.0,19.0,10.0,28.0,14.0,6.0,10.0,0.500000


In [21]:
df_final['points_per_game'] = df_final['points_per_game'].round(2)
df_final.head()

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,season,name,team,position,games_played,onIce_corsiPercentage,onIce_fenwickPercentage,I_F_goals,I_F_points,I_F_penalityMinutes,I_F_hits,I_F_takeaways,I_F_giveaways,Games_Missed,points_per_game
40,2012,Jack Hillen,WSH,D,23,0.49,0.50,3.0,9.0,14.0,10.0,2.0,13.0,0.0,0.39
56,2012,David Perron,STL,L,48,0.58,0.58,10.0,25.0,44.0,48.0,26.0,14.0,0.0,0.52
94,2012,Ryane Clowe,NYR,L,40,0.58,0.58,3.0,19.0,53.0,99.0,13.0,26.0,0.0,0.48
103,2012,Nicklas Grossmann,PHI,D,30,0.41,0.42,1.0,4.0,21.0,81.0,6.0,17.0,18.0,0.13
117,2012,Vladimir Tarasenko,STL,R,38,0.63,0.63,8.0,19.0,10.0,28.0,14.0,6.0,10.0,0.50


In [22]:

df_final = df_final.rename(columns={'onIce_corsiPercentage': 'corsi_percentage'})
df_final = df_final.rename(columns={'onIce_fenwickPercentage': 'fenwick_percentage'})
df_final = df_final.rename(columns={'I_F_goals': 'goals'})
df_final = df_final.rename(columns={'I_F_points': 'points'})
df_final = df_final.rename(columns={'I_F_penalityMinutes': 'penalty_minutes'})
df_final = df_final.rename(columns={'I_F_hits': 'hits'})
df_final = df_final.rename(columns={'I_F_takeaways': 'takeaways'})
df_final = df_final.rename(columns={'I_F_giveaways': 'giveaways'})
df_final = df_final.rename(columns={'Games_Missed': 'games_missed'})
df_final.head()

,season,name,team,position,games_played,corsi_percentage,fenwick_percentage,goals,points,penalty_minutes,hits,takeaways,giveaways,games_missed,points_per_game
40,2012,Jack Hillen,WSH,D,23,0.49,0.50,3.0,9.0,14.0,10.0,2.0,13.0,0.0,0.39
56,2012,David Perron,STL,L,48,0.58,0.58,10.0,25.0,44.0,48.0,26.0,14.0,0.0,0.52
94,2012,Ryane Clowe,NYR,L,40,0.58,0.58,3.0,19.0,53.0,99.0,13.0,26.0,0.0,0.48
103,2012,Nicklas Grossmann,PHI,D,30,0.41,0.42,1.0,4.0,21.0,81.0,6.0,17.0,18.0,0.13
117,2012,Vladimir Tarasenko,STL,R,38,0.63,0.63,8.0,19.0,10.0,28.0,14.0,6.0,10.0,0.50


In [23]:
df_final.dtypes

season                  int64
name                   object
team                   object
position               object
games_played            int64
corsi_percentage      float64
fenwick_percentage    float64
goals                 float64
points                float64
penalty_minutes       float64
hits                  float64
takeaways             float64
giveaways             float64
games_missed          float64
points_per_game       float64
dtype: object

In [24]:
df_final['goals'] = df_final['goals'].astype(int)
df_final['points'] = df_final['points'].astype(int)
df_final['penalty_minutes'] = df_final['penalty_minutes'].astype(int)
df_final['hits'] = df_final['hits'].astype(int)
df_final['takeaways'] = df_final['takeaways'].astype(int)
df_final['giveaways'] = df_final['giveaways'].astype(int)
df_final['games_missed'] = df_final['games_missed'].astype(int)
df_final['name'] = df_final['name'].astype(str)
df_final['team'] = df_final['team'].astype(str)
df_final['position'] = df_final['position'].astype(str)
df_final.dtypes


season                  int64
name                   object
team                   object
position               object
games_played            int64
corsi_percentage      float64
fenwick_percentage    float64
goals                   int64
points                  int64
penalty_minutes         int64
hits                    int64
takeaways               int64
giveaways               int64
games_missed            int64
points_per_game       float64
dtype: object

In [25]:
df_final.head()

,season,name,team,position,games_played,corsi_percentage,fenwick_percentage,goals,points,penalty_minutes,hits,takeaways,giveaways,games_missed,points_per_game
40,2012,Jack Hillen,WSH,D,23,0.49,0.50,3,9,14,10,2,13,0,0.39
56,2012,David Perron,STL,L,48,0.58,0.58,10,25,44,48,26,14,0,0.52
94,2012,Ryane Clowe,NYR,L,40,0.58,0.58,3,19,53,99,13,26,0,0.48
103,2012,Nicklas Grossmann,PHI,D,30,0.41,0.42,1,4,21,81,6,17,18,0.13
117,2012,Vladimir Tarasenko,STL,R,38,0.63,0.63,8,19,10,28,14,6,10,0.50


In [26]:
# Write the dataframe to postgres database
df_final.to_sql('players_of_interest', db, if_exists='replace',
          index=False)
sql1 = '''select * from clean_concussion_data;'''
cursor.execute(sql1)
for i in cursor.fetchall():
    print(i)
  
conn.commit()
conn.close()

(2012, 'BOS', 'F', 'Patrice Bergeron', 6)
(2012, 'BOS', 'F', 'Brad Marchand', 2)
(2012, 'BOS', 'F', 'Shawn Thornton', 2)
(2012, 'CAR', 'F', 'Chad LaRose', 10)
(2012, 'CAR', 'F', 'Alexander Semin', 2)
(2012, 'CAR', 'F', 'Jeff Skinner', 5)
(2012, 'CHI', 'D', 'Steve Montador', 26)
(2012, 'CBJ', 'D', 'James Wisniewski', 2)
(2012, 'DET', 'F', 'Patrick Eaves', 1)
(2012, 'EDM', 'F', 'Ben Eager', 9)
(2012, 'FLA', 'F', 'Eric Selleck', 16)
(2012, 'FLA', 'D', 'Mike Weaver', 1)
(2012, 'MTL', 'F', 'Rene Bourque', 21)
(2012, 'MTL', 'F', 'Brendan Gallagher', 3)
(2012, 'MTL', 'D', 'Raphael Diaz', 25)
(2012, 'NYR', 'F', 'Darroll Powe', 3)
(2012, 'OTT', 'F', 'David Dziurzynski', 7)
(2012, 'OTT', 'D', 'Mike Lundin', 8)
(2012, 'PHI', 'F', 'Daniel Briere', 10)
(2012, 'PHI', 'F', 'Wayne Simmonds', 3)
(2012, 'PHI', 'D', 'Marc-Andre Bourdon', 48)
(2012, 'PHI', 'D', 'Nicklas Grossmann', 18)
(2012, 'PHI', 'D', 'Kent Huskins', 6)
(2012, 'ARI', 'F', 'Rob Klinkhammer', 1)
(2012, 'PIT', 'F', 'Evgeni Malkin', 4)
(20

In [27]:
df_final.to_csv("../Resources/Players_of_Interest.csv", index = False)

In [28]:
df_final.to_json("../static/js/data.json", indent = 2)